In [1]:
from amplify import gen_symbols, BinaryPoly, IsingPoly
from amplify import sum_poly
from amplify.constraint import equal_to
import matplotlib.pyplot as plt
from amplify import Solver
from amplify.client import FixstarsClient
import matplotlib.ticker as ticker   
import os

In [2]:
def q_annealing(n, c):
    s = gen_symbols(IsingPoly, n+1, n+1)

    # 確定しているもののスピンを確定------------------------------------------
    for i in range(n):
        for j in range(n):
            if c[i][j] == "B":
                s[i][j] = 1
            if c[i][j] == "W":
                s[i][j] = -1

    # ハミルトニアン1 コスト項　反強磁性イジングモデル-----------------------------------
    f = 0
    for i in range(n):
        for j in range(n):
            if i != n-1:
                f += s[i][j] * s[i+1][j]
            if j != n-1:
                f += s[i][j] * s[i][j+1]

    # n=1のときや？が存在しないケースに対応するためにまったく関係ないスピンを追加
    # スピンは必ず-1になるので+1を足して打ち消し
    f += s[n][n] + 1 
                
    model = f  
    
    # 計算実行------------------------------------------------------------------------------------------
    client = FixstarsClient()
    with open("tokenfile.txt", "r") as f:
        token = f.readline()
    client.token = token
    client.parameters.timeout = 9000  # タイムアウト9秒
    solver = Solver(client)

    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Any one of constraints is not satisfied.")

    energy, values = result[0].energy, result[0].values
    
    return int((2*n*(n-1) -energy)/2)

In [3]:

# テストケース読み込み
# https://www.dropbox.com/sh/nx3tnilzqz7df8a/AACpUzqOotp665-GQMdAbHJua/ABC193/F?dl=0&subfolder_nav_tracking=1

files = os.listdir("ABC193F/in")
for file in files:
    with open("ABC193F/in/" + file, "r") as f:
        n = 

    with open("ABC193F/out/" + file, "r") as f:
        ans = f.readline()

    try:
        result = q_annealing(n)
        print(file, "Amplify結果 =", result, "正解 =", ans)
    except:
        pass


01_sample.txt Amplify結果 = 2 正解 = 2
02_sample.txt Amplify結果 = 4 正解 = 4
03_sample.txt Amplify結果 = 40 正解 = 40
04_hand.txt Amplify結果 = 0 正解 = 0
05_hand.txt Amplify結果 = 0 正解 = 0
06_small.txt Amplify結果 = 81 正解 = 81
07_small.txt Amplify結果 = 142 正解 = 142
08_small.txt Amplify結果 = 39 正解 = 39
09_small.txt Amplify結果 = 84 正解 = 84
10_small.txt Amplify結果 = 39 正解 = 39
11_small.txt Amplify結果 = 62 正解 = 62
12_small.txt Amplify結果 = 7 正解 = 7
13_small.txt Amplify結果 = 73 正解 = 73
14_small.txt Amplify結果 = 10 正解 = 10
15_small.txt Amplify結果 = 0 正解 = 0
16_small.txt Amplify結果 = 48 正解 = 48
17_small.txt Amplify結果 = 0 正解 = 0
18_large.txt Amplify結果 = 5535 正解 = 5535
19_large.txt Amplify結果 = 3690 正解 = 3690
20_large.txt Amplify結果 = 103 正解 = 103
21_large.txt Amplify結果 = 0 正解 = 0
22_large.txt Amplify結果 = 4442 正解 = 4442
23_large.txt Amplify結果 = 8550 正解 = 8550
24_large.txt Amplify結果 = 14699 正解 = 14702
25_large.txt Amplify結果 = 8635 正解 = 8635
26_large.txt Amplify結果 = 13750 正解 = 13750
27_large.txt Amplify結果 = 0 正解 = 0
28_max.tx